In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/Project_576/
!ls -la

/content/drive/My Drive/Colab Notebooks/Project_576
total 691490
-rw------- 1 root root     52727 Apr 27 19:49 Copy_of_LSTM_MultiLayer_BiDirectional_Shreiya.ipynb
-rw------- 1 root root     31564 Apr 26 22:29 Copy_of_NMT_2_Shreiya.ipynb
-rw------- 1 root root 154916665 Apr 27 03:26 decoder_lstm_7ep.pkl
-rw------- 1 root root 154916678 Apr 27 19:45 decoder_lstm_8ep.pkl
-rw------- 1 root root     38475 Apr 24 22:09 Doc_similarity_Shreiya.ipynb
-rw------- 1 root root  66279583 Apr 27 03:26 encoder_lstm_7ep.pkl
-rw------- 1 root root  66279583 Apr 27 19:44 encoder_lstm_8ep.pkl
-rw------- 1 root root  57338758 Apr 24 18:36 encoder_nmt.pkl
-rw------- 1 root root  45442087 Apr 26 21:42 hindencorp05.export.gz
-rw------- 1 root root  72209027 Apr 26 21:42 hindencorp05.plaintext
-rw------- 1 root root  23899723 Apr 26 21:42 hindencorp05.plaintext.gz
-rw------- 1 root root  66672230 Apr 26 21:42 hindencorp05.txt
-rw------- 1 root root      4472 Apr 26 21:42 README.txt


In [0]:
!curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11858/00-097C-0000-0023-625F-0{/README.txt,/hindencorp05.export.gz,/hindencorp05.plaintext.gz}

In [0]:
!gunzip -k hindencorp05.plaintext.gz

In [0]:
!cut -f 4- hindencorp05.plaintext > hindencorp05.txt

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    table = str.maketrans('', '', string.punctuation)
    s = s.translate(table)
    s = re.sub(r"[.!?]+", r" ", s)
    return s

In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('hindencorp05.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [0]:
MAX_LENGTH = 15


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [0]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'hin', False)
print(random.choice(pairs))

Reading lines...
Read 273885 sentence pairs
Trimmed to 182030 sentence pairs
Counting words...
Counted words:
eng 54452
hin 70097
['large mouse cursor', 'बड़ा माउस कर्सर']


In [0]:
%matplotlib inline
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers = 2, bidirectional = True)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        forward_output, backward_output = output[:, :, :self.hidden_size], output[:, :, self.hidden_size:]
        output = torch.cat((forward_output, backward_output))
        return output, hidden

    def initHidden(self):
        return (torch.zeros(4, 1, self.hidden_size, device=device), torch.zeros(4, 1, self.hidden_size, device=device))


In [0]:
%matplotlib inline
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
%matplotlib inline
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, num_layers = 2, bidirectional = True)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0][0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
       
        forward_output, backward_output = \
            output[:, :, :self.hidden_size], output[:, :, self.hidden_size:]
       
        output = torch.cat((forward_output, backward_output))
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return (torch.zeros(4, 1, self.hidden_size, device=device), torch.zeros(4, 1, self.hidden_size, device=device))

In [0]:
def indexesFromSentence(lang, sentence):
    %matplotlib inline
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
%matplotlib inline
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    %matplotlib inline

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
       
        encoder_outputs[ei] = encoder_output[0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [0]:
%matplotlib inline
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
random.choice(pairs)

['united states customary units', 'संयुक्त राज्य परंपरागत इकाइयाँ']

In [0]:
len(pairs)

182030

In [0]:
%matplotlib inline
def trainIters(encoder, decoder, n_epochs, learning_rate, print_every=50000, plot_every=50000):
    %matplotlib inline
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(pairs[i]) for i in range(len(pairs))]
    criterion = nn.NLLLoss()
    
    n_iters = len(pairs)
    for e in range(1, n_epochs + 1):
        for iter in range(1, len(pairs) + 1):
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    showPlot(plot_losses)


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    %matplotlib inline
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    %matplotlib inline
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
           
            encoder_outputs[ei] += encoder_output[0, 0]
                     

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=5):
    from nltk.translate.bleu_score import sentence_bleu
    %matplotlib inline
    for i in range(n):
        pair = random.choice(pairs)
        print('English: ', pair[0])
        print('Actual: ', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('Predicted: ', output_sentence)
        act = pair[1].split()
        pred = output_sentence.split()
        print('Cumulative 1-gram: %f' % sentence_bleu([act], pred, weights=(1, 0, 0, 0)))
        print('Cumulative 2-gram: %f' % sentence_bleu([act], pred, weights=(0.5, 0.5, 0, 0)))
        print('Cumulative 3-gram: %f' % sentence_bleu([act], pred, weights=(0.33, 0.33, 0.33, 0)))
        print('Cumulative 4-gram: %f' % sentence_bleu([act], pred, weights=(0.25, 0.25, 0.25, 0.25)))
        score = sentence_bleu([act], pred)
        print(score)
        print('')

In [0]:
%matplotlib inline
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)


encoder1.load_state_dict(torch.load('encoder_lstm_8ep.pkl'))
attn_decoder1.load_state_dict(torch.load('decoder_lstm_8ep.pkl'))

trainIters(encoder1, attn_decoder1, 1, print_every=20000, learning_rate=0.0001)

torch.save(encoder1.state_dict(),'encoder_lstm_9ep.pkl')
torch.save(attn_decoder1.state_dict(),'decoder_lstm_9ep.pkl')


In [0]:
evaluateRandomly(encoder1, attn_decoder1)

In [0]:
#Flickr8 Dataset Img Captioning 

sent1 = " Two dogs are running together through the grass "
sent2 = " man is on a big wave"
sent3 = " man in a helmet rides a bike prediction"
#sent4 = " man on a surfboard "
sent5 = "woman and a woman are smiling and smiling"
sent6 = "group of people are sitting on a bench"
sent7 = " football player in a red uniform is running "
sent8 = "man in a blue shirt and blue shorts is on the street"
sent9 = "two brown dogs are playing in the snow"




output_words1, attentions1 = evaluate(encoder1, attn_decoder1, sent1.lower())
output_words2, attentions2 = evaluate(encoder1, attn_decoder1, sent2.lower())
output_words3, attentions3 = evaluate(encoder1, attn_decoder1, sent3.lower())
#output_words4, attentions4 = evaluate(encoder1, attn_decoder1, sent4.lower())
output_words5, attentions5 = evaluate(encoder1, attn_decoder1, sent5.lower())
output_words6, attentions6 = evaluate(encoder1, attn_decoder1, sent6.lower())
output_words7, attentions7 = evaluate(encoder1, attn_decoder1, sent7.lower())
output_words8, attentions8 = evaluate(encoder1, attn_decoder1, sent8.lower())
output_words9, attentions9 = evaluate(encoder1, attn_decoder1, sent9.lower())

output_sentence1 = ' '.join(output_words1)
output_sentence2 = ' '.join(output_words2)
output_sentence3 = ' '.join(output_words3)
#output_sentence4 = ' '.join(output_words4)
output_sentence5 = ' '.join(output_words5)
output_sentence6 = ' '.join(output_words6)
output_sentence7 = ' '.join(output_words7)
output_sentence8 = ' '.join(output_words8)
output_sentence9 = ' '.join(output_words9)


print('Predicted1: ', output_sentence1)
print('Predicted2: ', output_sentence2)
print('Predicted3: ', output_sentence3)
#print('Predicted4: ', output_sentence4)
print('Predicted5: ', output_sentence5)
print('Predicted6: ', output_sentence6)
print('Predicted7: ', output_sentence7)
print('Predicted8: ', output_sentence8)
print('Predicted9: ', output_sentence9)

In [0]:
sent1 = " Two dogs are running together through the grass "
sent2 = " man is on a big wave"
sent3 = " man in a helmet rides a bike prediction"
#sent4 = " man on a surfboard "
sent5 = "woman and a woman are smiling and smiling"
sent6 = "group of people are sitting on a bench"
sent7 = " football player in a red uniform is running "
sent8 = "man in a blue shirt and blue shorts is on the street"
sent9 = "two brown dogs are playing in the snow"

In [0]:
#sent1 = "Two dogs are running"
#sent2 = "man in red shirt and black pants is standing on wooden bench"
#sent3 = "man in jacket is sitting on rock"
#sent4 = "man is battling waves in purple boat while wearing rain gear"
#sent5 = "man in yellow suit is holding up cup while standing in snow"
#sent6 = "blonde horse and blonde girl in black are staring at fire in barrel"
#sent7 = "Two people and two horses watching fire"
#sent8 = "small pale bird down to examine and small white bird looks at small object"

#output_words1, attentions1 = evaluate(encoder1, attn_decoder1, sent1.lower())
#output_words2, attentions2 = evaluate(encoder1, attn_decoder1, sent2.lower())
#output_words3, attentions3 = evaluate(encoder1, attn_decoder1, sent3.lower())
#output_words4, attentions4 = evaluate(encoder1, attn_decoder1, sent4.lower())
#output_words5, attentions5 = evaluate(encoder1, attn_decoder1, sent5.lower())
#output_words6, attentions6 = evaluate(encoder1, attn_decoder1, sent6.lower())
#output_words7, attentions7 = evaluate(encoder1, attn_decoder1, sent7.lower())
#output_words8, attentions8 = evaluate(encoder1, attn_decoder1, sent8.lower())

#output_sentence1 = ' '.join(output_words1)
#output_sentence2 = ' '.join(output_words2)
#output_sentence3 = ' '.join(output_words3)
#output_sentence4 = ' '.join(output_words4)
#output_sentence5 = ' '.join(output_words5)
#output_sentence6 = ' '.join(output_words6)
#output_sentence7 = ' '.join(output_words7)
#output_sentence8 = ' '.join(output_words8)


#print('Predicted1: ', output_sentence1)
#print('Predicted2: ', output_sentence2)
#print('Predicted3: ', output_sentence3)
#print('Predicted4: ', output_sentence4)
#print('Predicted5: ', output_sentence5)
#print('Predicted6: ', output_sentence6)
#print('Predicted7: ', output_sentence7)
#print('Predicted8: ', output_sentence8)


In [0]:
sent1 = "Two dogs are running"
sent2 = "man in red shirt and black pants is standing on wooden bench"
sent3 = "man in jacket is sitting on rock"
sent4 = "man is battling waves in purple boat while wearing rain gear"
sent5 = "man in yellow suit is holding up cup while standing in snow"
sent6 = "blonde horse and blonde girl in black are staring at fire in barrel"
sent7 = "Two people and two horses watching fire"
sent8 = "small pale bird down to examine and small white bird looks at small object"

In [0]:
#FLickr8Dataset for LSTM


sent1 = " man and woman sit on a car and watch the road"
sent2 = " dog runs through tall grass "
sent3 = " A brown dog playing in mud"
sent4 = " dog is playing in water "
sent5 = " woman wearing a pearl necklace "
sent6 = " couple sits on a car "
sent7 = " man and woman sitting on a parked car "
sent8 = " dog in a field "


output_words1, attentions1 = evaluate(encoder1, attn_decoder1, sent1.lower())
output_words2, attentions2 = evaluate(encoder1, attn_decoder1, sent2.lower())
output_words3, attentions3 = evaluate(encoder1, attn_decoder1, sent3.lower())
output_words4, attentions4 = evaluate(encoder1, attn_decoder1, sent4.lower())
output_words5, attentions5 = evaluate(encoder1, attn_decoder1, sent5.lower())
output_words6, attentions6 = evaluate(encoder1, attn_decoder1, sent6.lower())
output_words7, attentions7 = evaluate(encoder1, attn_decoder1, sent7.lower())
output_words8, attentions8 = evaluate(encoder1, attn_decoder1, sent8.lower())

output_sentence1 = ' '.join(output_words1)
output_sentence2 = ' '.join(output_words2)
output_sentence3 = ' '.join(output_words3)
output_sentence4 = ' '.join(output_words4)
output_sentence5 = ' '.join(output_words5)
output_sentence6 = ' '.join(output_words6)
output_sentence7 = ' '.join(output_words7)
output_sentence8 = ' '.join(output_words8)


print('Predicted1: ', output_sentence1)
print('Predicted2: ', output_sentence2)
print('Predicted3: ', output_sentence3)
print('Predicted4: ', output_sentence4)
print('Predicted5: ', output_sentence5)
print('Predicted6: ', output_sentence6)
print('Predicted7: ', output_sentence7)
print('Predicted8: ', output_sentence8)


In [0]:
sent1 = " man and woman sit on a car and watch the road"
sent2 = " dog runs through tall grass "
sent3 = " A brown dog playing in mud"
sent4 = " dog is playing in water "
sent5 = " woman wearing a pearl necklace "
sent6 = " couple sits on a car "
sent7 = " man and woman sitting on a parked car "
sent8 = " dog in a field "

In [0]:
#Hindencorp sentences


sent1 = "The ending portion of these Vedas is called Upanishad"
sent2 = "Please ensure that you use the appropriate form"
sent3 = "General Insurance Corporation of India"
sent4 = "There are also financial reforms in rural China"
sent5 = "Twenty of the years from 1860 to 1908 were years of famine"
sent6 = "This change has contributed"
sent7 = "Default Mail Client "
sent8 = "File Management Preferences"


output_words1, attentions1 = evaluate(encoder1, attn_decoder1, sent1.lower())
output_words2, attentions2 = evaluate(encoder1, attn_decoder1, sent2.lower())
output_words3, attentions3 = evaluate(encoder1, attn_decoder1, sent3.lower())
output_words4, attentions4 = evaluate(encoder1, attn_decoder1, sent4.lower())
output_words5, attentions5 = evaluate(encoder1, attn_decoder1, sent5.lower())
output_words6, attentions6 = evaluate(encoder1, attn_decoder1, sent6.lower())
output_words7, attentions7 = evaluate(encoder1, attn_decoder1, sent7.lower())
output_words8, attentions8 = evaluate(encoder1, attn_decoder1, sent8.lower())

output_sentence1 = ' '.join(output_words1)
output_sentence2 = ' '.join(output_words2)
output_sentence3 = ' '.join(output_words3)
output_sentence4 = ' '.join(output_words4)
output_sentence5 = ' '.join(output_words5)
output_sentence6 = ' '.join(output_words6)
output_sentence7 = ' '.join(output_words7)
output_sentence8 = ' '.join(output_words8)


print('Predicted1: ', output_sentence1)
print('Predicted2: ', output_sentence2)
print('Predicted3: ', output_sentence3)
print('Predicted4: ', output_sentence4)
print('Predicted5: ', output_sentence5)
print('Predicted6: ', output_sentence6)
print('Predicted7: ', output_sentence7)
print('Predicted8: ', output_sentence8)


In [0]:
sent1 = "The ending portion of these Vedas is called Upanishad"
sent2 = "Please ensure that you use the appropriate form"
sent3 = "General Insurance Corporation of India"
sent4 = "There are also financial reforms in rural China"
sent5 = "Twenty of the years from 1860 to 1908 were years of famine"
sent6 = "This change has contributed"
sent7 = "Default Mail Client "
sent8 = "File Management Preferences"